In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import glob
import os

In [2]:
# Redefinir pd.read_csv e pd.concat para as funções originais
pd.read_csv = pd.io.parsers.read_csv
pd.concat = pd.core.reshape.concat.concat

In [3]:
normal = "C:/Mafaulda/normal"

In [4]:
arquivos_csv = glob.glob(os.path.join(normal, "*.csv"))

In [5]:
# Carregar e concatenar todos os arquivos CSV em um único DataFrame
df_normal = pd.concat((pd.read_csv(f) for f in arquivos_csv), ignore_index=True)

# Exibir as primeiras linhas do DataFrame resultante
print(df_normal.head())

ValueError: No objects to concatenate

In [ ]:
print("Tamanho data_dir:", len(normal))
print("Tamanho do dataset:", df_normal.shape)
print("--------------------")
df_normal.info()

In [ ]:
print(df_normal.columns)


### Trabalhando o arquivo Normal 28.8768.csv:

In [ ]:
# Caminho para o arquivo CSV específico
arquivo_01_csv = "c:/Mafaulda/normal/28.8768.csv"

# Carregar o arquivo CSV específico
df_arquivo_01_csv = pd.read_csv(arquivo_01_csv)

# Exibir as primeiras linhas do DataFrame resultante
print(df_arquivo_01_csv.head())

### Colunas:

- column 1: tachometer signal that allows to estimate rotation frequency - TS;

- columns 2 to 4: underhang bearing accelerometer - UBA (axial, radiale tangential direction);

- columns 5 to 7: overhang bearing accelerometer - OBA (axial, radiale tangential direction);

- column 8 - microphone - MIC.

In [9]:
nomes_colunas = ['TS', 'UBA_01', 'UBA_02', 'UBA_03', 'OBA_01', 'OBA_02', 'OBA_03', 'MIC']
arquivo_01_csv = "c:/Mafaulda/normal/28.8768.csv"
df_arquivo_01_csv = pd.read_csv(arquivo_01_csv, header=None, names=nomes_colunas)

In [ ]:
print(df_arquivo_01_csv.head())
print(df_arquivo_01_csv.describe())

In [ ]:
import matplotlib.pyplot as plt
df_arquivo_01_csv['TS'].hist(bins=30)
plt.xlabel('Valores')
plt.ylabel('Frequência')
plt.title('Distribuição de Frequências')
plt.show()

In [ ]:
df_arquivo_01_csv.info()

In [ ]:
df_arquivo_01_csv.isnull().sum()

In [ ]:
df_arquivo_01_csv.corr()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(8, 12))
axes = axes.flatten()
for i, col in enumerate(df_arquivo_01_csv.columns[:8]):  # Limitando às primeiras 9 colunas
    df_arquivo_01_csv[col].hist(ax=axes[i], bins=40)
    axes[i].set_title(col)

#df_arquivo_01_csv.hist()
plt.show()

### Abordando o problema de forma não supervisionada:

- Uso de autoencoder tanto no SBM como no AAKR, adapatados para funcionar como modelos de aprendizado não supervisionado. Prevendo padrões ou anomalias nos dados (análise auto-associativa):

In [16]:
df = df_arquivo_01_csv

X = df[['TS', 'UBA_01', 'UBA_02', 'UBA_03', 'OBA_01', 'OBA_02', 'OBA_03', 'MIC']]

In [ ]:
# Exemplo de dados para X (substitua pelos seus dados reais)
X = np.random.rand(40000, 1)  # Exemplo de 1000 amostras, 1 característica

X = X.reshape((-1, 1))

# Separação desenvolvimento (30) e teste (70)
X_dev, X_test = train_test_split(X, test_size=0.8, random_state=42)

# Separação treino (80) e validação (20) - Hold-out
X_train, X_val, = train_test_split(X_dev, test_size=0.8, random_state=42)

# Normalizando dados para treino, validação e teste
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

# Exibir as formas dos conjuntos de dados
print("Shape do conjunto de treino:", X_train.shape)
print("Shape do conjunto de validação:", X_val.shape)
print("Shape do conjunto de teste:", X_test.shape)

In [ ]:
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import Dense, Dropout

# Autoencoder simples para SBM
def create_sbm_autoencoder(input_shape):
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dropout(0.30),  # Dropout após a primeira camada
        Dense(32, activation='relu'),  # Encoder
        Dropout(0.30),  # Dropout após o Encoder
        Dense(64, activation='relu'),  # Decoder
        Dropout(0.30),  # Dropout após o Decoder
        Dense(input_shape)  # Saída de mesma dimensão que a entrada
    ])
    model.compile(optimizer='adam', loss='mse')  # Usar MAE para medir o erro
    return model


# Callback personalizado para acompanhar a acurácia
class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        accuracy = 1 - loss
        val_accuracy = 1 - val_loss
        print(f"Epoch {epoch+1}: Perda Treinamento: {loss:.4f}, Acurácia Treinamento: {accuracy:.4f}, Perda Validação: {val_loss:.4f}, Acurácia Validação: {val_accuracy:.4f}")
        
        

# Criar o autoencoder e atribuir à variável 'sbm_autoencoder'
sbm_autoencoder = create_sbm_autoencoder(X_train.shape[1])

# Exibir o sumário do modelo
sbm_autoencoder.summary()



- A acurácia de um autoencoder geralmente é medida com base no erro de reconstrução:

In [ ]:

# Treinar o modelo SBM e armazenar o histórico de treinamento
history = sbm_autoencoder.fit(
    X_train, X_train,
    epochs=40,
    batch_size=256,
    validation_data=(X_val, X_val),
    callbacks=[AccuracyCallback()],  # "AccuracyCallback" com 'C' maiúsculo
    verbose=1
)

# Fazer previsões com o Autoencoder SBM
X_pred_sbm = sbm_autoencoder.predict(X_test)
sbm_value = np.mean(np.abs(X_test - X_pred_sbm))  # Erro médio absoluto como SBM

# Calcular "acurácia" inversamente proporcional ao erro de reconstrução (quanto menor o erro, melhor a acurácia)
accuracy = 1 - sbm_value

# Exibir resultado SBM
print(f"SBM (MSE - Erro de Reconstrução): {sbm_value:.4f}")
print(f"Acurácia (1 - Erro de Reconstrução): {accuracy:.4f}")

In [ ]:
# Plotar curvas de perda (training e validation loss)
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Curvas de Perda (Training vs Validation)')
plt.xlabel('Épocas')
plt.ylabel('Perda (Loss)')
plt.legend()
plt.grid(True)
plt.show()

# Adicionar curvas de "acurácia" baseada no erro de reconstrução (quanto menor o erro, maior a acurácia)
train_accuracy = 1 - np.array(history.history['loss'])
val_accuracy = 1 - np.array(history.history['val_loss'])

# Plotar curvas de "acurácia"
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Curvas de Acurácia (Baseada no Erro de Reconstrução)')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.show()